## Rerun imports and initialization

In [ ]:
from collections import namedtuple
from math import cos, sin, tau
import math

import numpy as np

import rerun as rr

viewer.init("cube")

## Optional: start a local web-viewer server

By default, Rerun will use a copy of the viewer hosted at [https://app.rerun.io](https://app.rerun.io).
This is generally preferable as it will work more seamlessly even if you
are connected to a notebook instance on a remote machine. However there
are some cases where this won't work such as running from source, or
using your notebook in an offline environment.

In these cases you can start a local viewer server by uncommenting the following
line:

In [ ]:
# viewer.start_web_viewer_server()

## Helper to create the colored cube

This is the same as the color cube demo from `rerun -m rerun_demo`, but the code
is repeated here for context.

In [ ]:
ColorGrid = namedtuple("ColorGrid", ["positions", "colors"])


def build_color_grid(x_count=10, y_count=10, z_count=10, twist=0):
    """
    Create a cube of points with colors.

    The total point cloud will have x_count * y_count * z_count points.

    Parameters
    ----------
    x_count, y_count, z_count:
        Number of points in each dimension.
    twist:
        Angle to twist from bottom to top of the cube

    """

    grid = np.mgrid[
        slice(-10, 10, x_count * 1j),
        slice(-10, 10, y_count * 1j),
        slice(-10, 10, z_count * 1j),
    ]

    angle = np.linspace(-float(twist) / 2, float(twist) / 2, z_count)
    for z in range(z_count):
        xv, yv, zv = grid[:, :, :, z]
        rot_xv = xv * cos(angle[z]) - yv * sin(angle[z])
        rot_yv = xv * sin(angle[z]) + yv * cos(angle[z])
        grid[:, :, :, z] = [rot_xv, rot_yv, zv]

    positions = np.vstack([xyz.ravel() for xyz in grid])

    colors = np.vstack(
        [
            xyz.ravel()
            for xyz in np.mgrid[
                slice(0, 255, x_count * 1j),
                slice(0, 255, y_count * 1j),
                slice(0, 255, z_count * 1j),
            ]
        ]
    )

    return ColorGrid(positions.T, colors.T.astype(np.uint8))

## Start a new recording

To start a new recording all you need to do is call viewer.memory_recording().

In [ ]:
rec = viewer.memory_recording()

## Showing the recording

At any point you can show this recording by returning it as the last item in the cell.
In this case the recording simply does not have any data in it yet.

In [ ]:
rec

## Logging some data

Now we can create some data and add it to the recording before we show it again.

In [ ]:
STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    viewer.set_time_sequence("step", t)
    cube = build_color_grid(10, 10, 10, twist=twists[t])
    viewer.log_points("cube", positions=cube.positions, colors=cube.colors, radii=0.5)

rec

## Adjusting the view

The recording also as a `show` method that lets you adjust properties such as width and height.
In the future this will support additional blueprint and layout options.

In [ ]:
rec.show(width=400, height=400)

## Stating a new recording

You can always start another recording by calling `viewer.memory_recording()` again.

In [ ]:
rec2 = viewer.memory_recording()

STEPS = 1
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    viewer.set_time_sequence("step", t)
    cube = build_color_grid(50, 50, 50, twist=twists[t])
    viewer.log_points("cube", positions=cube.positions, colors=cube.colors, radii=0.5)

rec2